# Covid-19 Dashboard

In [39]:
# Import necessary libraries
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import pickle
import time
from uk_covid19 import Cov19API

# Include graphs in notebook
%matplotlib inline
# Make graphs larger
plt.rcParams['figure.dpi'] = 100

# Load pickle files
with open('timeseriesdf.pkl', 'rb') as f:
    df = pickle.load(f)

# Function to create a panda date from the date string
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

# Function to format the data into a data frame
def wrangle_data(rawdata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    datalist=rawdata['data']
    # Extract all dates from the datalist and sort them in order
    dates=[dictionary['date'] for dictionary in datalist]
    dates.sort()
    # Create a panda date for the first date in the series
    startdate=parse_date(dates[0])
    # Create a panda date for the last date in the series
    enddate=parse_date(dates[-1])
    # Create a list of dates from the start date to the end date
    index=pd.date_range(startdate, enddate, freq='D')
    # Create a timeseries of dates with the columns
    timeseriesdf=pd.DataFrame(index=index, columns=['cases', 'hospital', 'deaths'])
    # Update the timeseries with data for the columns ['cases', 'hospital', 'deaths']
    for entry in datalist:
        date=parse_date(entry['date'])
        for column in ['cases', 'hospital', 'deaths']:
            if pd.isna(timeseriesdf.loc[date, column]): 
                # Replace none values with 0 in our data 
                value= float(entry[column]) if entry[column]!=None else 0.0
                # Update the column with the correct data
                timeseriesdf.loc[date, column]=value
    # Fill in any empty values with 0s
    timeseriesdf.fillna(0.0, inplace=True)
    return timeseriesdf

# Function to call the API
def access_api():
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
    # The definiton of the filter and structure of the API response 
    filters = ['areaType=overview']
    structure = {
        "date": "date",
        "cases": "newCasesByPublishDate",
        "hospital": "newAdmissions",
        "deaths": "cumDeaths28DaysByDeathDateRate"
        }
    # Create an API object
    api = Cov19API(filters=filters, structure=structure)
    # Get the API rsponse
    timeseries=api.get_json()
    # Return data read from the API
    return(timeseries)

# Function to trigger access_api function when the refresh button is clicked 
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Try calling the api
    try:
        apidata=access_api()
        # Format the data for plotting
        global df
        df=wrangle_data(apidata)
        # Save the outputs to the pickle file for when the dashboard is next viewd
        df.to_pickle("timeseriesdf.pkl")
        # Trigger the graph to refresh
        refresh_graph()
        # Update the Button to show the API was successful
        apibutton.description='Successful!'
        apibutton.icon="check"
        apibutton.disabled=True
        time.sleep(3)
        # Reset the Button 
        apibutton.description='Refresh Data'
        apibutton.icon="refresh"
        apibutton.disabled=False
    except:
        # Update the Button to show the API failed
        apibutton.description='Error'
        apibutton.icon="fa-exclamation-triangle"
        apibutton.button_style='warning'
        apibutton.disabled=True
        time.sleep(3)
        # Reset the Button
        apibutton.description='Refresh Data'
        apibutton.icon="refresh"
        apibutton.button_style='primary'
        apibutton.disabled=False
    
apibutton=wdg.Button(
    description='Refresh Data', # you may want to change this...
    disabled=False,
    button_style='primary', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Click to refresh the data in the graph",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='refresh'
)

# remember to register your button callback function with the button
apibutton.on_click(api_button_callback) # the name of your function inside these brackets

## Time Series Graph of Cases, Hospitalisations and Deaths

Cases relates to daily case relates

Hospital relates to new hospital admissions

Deaths relates to cumulative deaths within 28 days of positive test by death date per 100k resident population

Please refresh the data for the latest view of the graph

In [41]:
# Create a function to flick between log scale and columns displayed on the graph
def timeseries_graph(gcols, gscale):
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        df[list(gcols)].plot(logy=logscale)
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

# Create a widget for the user to select columns
series=wdg.SelectMultiple(
    options=['cases', 'hospital', 'deaths'],
    value=['cases', 'hospital', 'deaths'],
    rows=3,
    description='Stats:',
    disabled=False
)

# Create a button to allow users to flick between log and linear displays
scale=wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False
)

def refresh_graph():
    """ We change the value of the widget in order to force a redraw of the graph;
    this is useful when the data have been updated. This is a bit of a gimmick; it
    needs to be customised for one of your widgets. """
    current=scale.value
    if current==scale.options[0]:
        other=scale.options[1]
    else:
        other=scale.options[0]
    scale.value=other # forces the redraw
    scale.value=current # now we can change it back"""

    
controls=wdg.VBox([series, scale, apibutton])
    
graph=wdg.interactive_output(timeseries_graph, {'gcols': series, 'gscale': scale})

form=wdg.HBox([graph, controls])    
display(form)

**Author and Copyright Notice** (C) Guy Hunt, 2020 (g.p.w.hunt@se20.qmul.ac.uk). All rights reserved.: *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*